In [3]:
from db_commands import connect_db
import duckdb

conn = connect_db("olist.db")

 ! Database connected: olist.db


In [4]:
import sqlparse

def detect_dml_statements(content: str) -> list[dict[str, str]]:
    # This list covers DDL, DML, DCL, and TCL
    forbidden_types = {
        'INSERT', 'UPDATE', 'DELETE', 'DROP', 'CREATE', 
        'ALTER', 'TRUNCATE', 'GRANT', 'REVOKE', 'MERGE', 
        'COMMIT'
    }
    
    found_statements = []
    
    # Split string into individual SQL statements
    parsed = sqlparse.parse(content)
    
    for statement in parsed:
        # Get the first real token (e.g., 'SELECT', 'UPDATE', etc.)
        # sqlparse ignores comments and whitespace automatically
        root_keyword = statement.get_type()
        
        # Check if the main command is in our list
        if root_keyword in forbidden_types:
            found_statements.append({
                "statement": root_keyword,
                "full_query": str(statement).strip()
            })
            
        # Also check for sub-commands (like a DELETE inside a TRIGGER or a block)
        # We can iterate through tokens to find keywords inside the query
        else:
            for token in statement.flatten():
                if token.is_keyword and token.value.upper() in forbidden_types:
                    found_statements.append({
                        "statement": token.value.upper(),
                        "full_query": "Detected inside sub-query or block"
                    })
                    break # Avoid duplicate entries for the same query

    return found_statements

# --- Testing the difference ---
sql_test = """
GRANT SELECT, INSERT ON employees TO read_only_user;
"""

print(detect_dml_statements(sql_test)) 
# Result: [] (Correct! It ignored the word in the string and the word in the comment)

[{'statement': 'GRANT', 'full_query': 'Detected inside sub-query or block'}]


In [5]:
test_queries = [
    ("INSERT statement", "INSERT INTO employees (first_name, last_name, hire_date) VALUES ('Alice', 'Smith', '2023-01-15');"),
    ("UPDATE statement", "UPDATE products SET price = price * 1.10 WHERE category = 'Electronics';"),
    ("DELETE statement", "DELETE FROM orders WHERE order_date < '2022-01-01';"),
    ("MERGE statement", """
        MERGE INTO inventory AS target
        USING staging_inventory AS source
        ON (target.product_id = source.product_id)
        WHEN MATCHED THEN
            UPDATE SET target.quantity = source.quantity
        WHEN NOT MATCHED THEN
            INSERT (product_id, quantity) VALUES (source.product_id, source.quantity);
    """),
    ("TRUNCATE statement", "TRUNCATE TABLE logs;"),
    ("CREATE TABLE (DDL)", "CREATE TABLE users (user_id INT PRIMARY KEY, username VARCHAR(50) UNIQUE, created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP);"),
    ("DROP TABLE (DDL)", "DROP TABLE temp_data;"),
    ("GRANT (DCL)", "GRANT SELECT, INSERT ON employees TO read_only_user;"),
    ("COMMIT (TCL)", "COMMIT;"),
    ("SELECT with DELETE in string", "SELECT * FROM customers WHERE notes = 'This is an UPDATE on a DELETE task.'; -- safe SELECT"),
    ("SELECT with UPDATE in comment", "SELECT COUNT(*) FROM logs; -- UPDATE: need to check this later"),
    ("Complex statement with multiple keywords", "CREATE VIEW v_recent_orders AS SELECT order_id, customer_id FROM orders WHERE order_date >= NOW() - INTERVAL '7 days'; -- This is a SELECT statement, but CREATE is DDL")
]

print("--- Running Test Suite ---")
for name, query in test_queries:
    print(f"\nTesting: {name}")
    print(f"Query: \n{query}")
    results = detect_dml_statements(query)
    if results:
        print("Detected Statements:")
        for res in results:
            print(f"  - {res['statement']} (in query context: '{res['full_query'][:50]}...')")
    else:
        print("  (No forbidden statements detected)")

--- Running Test Suite ---

Testing: INSERT statement
Query: 
INSERT INTO employees (first_name, last_name, hire_date) VALUES ('Alice', 'Smith', '2023-01-15');
Detected Statements:
  - INSERT (in query context: 'INSERT INTO employees (first_name, last_name, hire...')

Testing: UPDATE statement
Query: 
UPDATE products SET price = price * 1.10 WHERE category = 'Electronics';
Detected Statements:
  - UPDATE (in query context: 'UPDATE products SET price = price * 1.10 WHERE cat...')

Testing: DELETE statement
Query: 
DELETE FROM orders WHERE order_date < '2022-01-01';
Detected Statements:
  - DELETE (in query context: 'DELETE FROM orders WHERE order_date < '2022-01-01'...')

Testing: MERGE statement
Query: 

        MERGE INTO inventory AS target
        USING staging_inventory AS source
        ON (target.product_id = source.product_id)
        WHEN MATCHED THEN
            UPDATE SET target.quantity = source.quantity
        WHEN NOT MATCHED THEN
            INSERT (product_id, quantity)

In [8]:
df = conn.sql("""
SELECT product_category_name_english, COUNT(DISTINCT product_id) AS total_products 
FROM products 
JOIN product_category_translation ON products.product_category_name = product_category_translation.product_category_name 
GROUP BY product_category_name_english;

"""
)
print(df)

┌─────────────────────────────────────────┬────────────────┐
│      product_category_name_english      │ total_products │
│                 varchar                 │     int64      │
├─────────────────────────────────────────┼────────────────┤
│ signaling_and_security                  │             93 │
│ art                                     │             55 │
│ construction_tools_construction         │            400 │
│ luggage_accessories                     │            349 │
│ food_drink                              │            104 │
│ books_technical                         │            123 │
│ fashion_male_clothing                   │             95 │
│ books_imported                          │             31 │
│ furniture_bedroom                       │             45 │
│ industry_commerce_and_business          │             68 │
│     ·                                   │              · │
│     ·                                   │              · │
│     ·                 